# <center>Record Linkage</center>
## <center>Vince Marinelli<br>CPSC548 Spring 2025</center>

Record Linkage refers to the task of linking records using a matching set of attributes shared by the two records. For example, we often need to match a list customers with a list of payees, or a list of patients with a list of users of a specific medication, etc. Another common example is the identification and removal of duplicate entries within a dataset. Record Linkage has also been referred to as data linkage, entity resolution, and data matching (<ins>Christen. 2019</ins>).

In the simplest case, we have a shared attribute that represents a unique identifier for the record (e.g. SSN, TIN, ISBN, etc). In this case we may assume that this identifier absolutely and completely identities the unique entity. Unfortunately, outside of the world of controlled schemas this is rarely true. More often, we need to use the full set of attributes that can be mapped between the two rows to develop a level of confidence that the two rows are the same. The attributes used to match are referred to as Quasi-Identifiers (QIDs). Each QID contributes a specific weight to the overall match probability that is proportional to the amount of information that the QID contributes. For example, a matching street address contributes a larger weight than a matching marital status because the street address has a higher cardinality / lower match probability then marital status.

The seminal work on record matching, **A Theory for Record Linkage**. was published in 1969 by Fellegi and Sunter. In the paper, the authors lay out a statistical theoretical basis for optimal record matching. It was later shown that this statistical basis is effectively the same as a Naive Baisian Classifer with the conditional independence assumption relaxed (<ins>Winkler, 2012</ins>). These statistical models remain mainstream today but are being challenged by newer models that use other types of classifiers.

In his 2012 book **Data Matching**, Peter Christen lays out the basic process that has historically been followed. The process contains the following steps (<ins>Christen, 2012, pp. 24-35</ins>):

1. Preprocessing - scrub and format record sets to be compared
2. Indexing - match each record from one data to records in the comparison data set
3. Comparison - compute similarity between a record and all possible comparators
4. Classification - classify records as matches or non-matches (or possible matches in some cases)
5. Evaluation - review the performance of the classification

Before delving into these steps in more detail, we will first review the Python library we will be using both for test data and for the execution of many of the above steps.

### Using the Python Record Linkage Toolkit (PRLT)
The [Python Record Linkage Toolkit](https://recordlinkage.readthedocs.io/en/latest/index.html) is a PyPi-hosted library that implements a standard set of record linkage activities. It includes tools for preprocessing data, indexing, comparison, classification and evaluation. The toolkit also contains several datasets that can be used to train and test algorithms.

First, we'll import all the modules that we'll be using in this workbook, including PRLT.

> NOTE: Some modules used in this project are not compatible with Python versions 3.12 and higher. It's required that we use a Python 3.11 Virtual Enviroment for this project.

In [1]:
import numpy as np
import pandas as pd
import pycountry
import re
import recordlinkage
import swifter
import time
import us

from concurrent.futures import ProcessPoolExecutor
from datasketch import MinHash, LeanMinHash, MinHashLSH
from nameparser import HumanName
from nameparser.config import Constants
from recordlinkage.datasets import load_febrl4
from recordlinkage.base import BaseIndexAlgorithm
from recordlinkage.preprocessing import phonetic

U:\Users\v_mar\OneDrive\Documents\KU\CPSC548\FinalProject\record_linkage\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Now that the modules have been imported, we will load data from the Freely Extensible Biomedical Record Linkage (febrl) package which is included in PRLT. Calling the `load_febrl4` method returns two datasets - one which contains all original records and a second that contains copies of the records from the first dataset, with duplicates included. A third output, a Pandas MultiIndex, can optionally be returned that contains the actual map from rows in the first dataframe to rows in the second (<ins>de Bruin, 2023</ins>). We'll retrieve all three objects and display them.

In [2]:
dfA, dfB, miTrueLinks = load_febrl4(return_links=True)
dfA

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-1070-org,michaela,neumann,8,stanley street,miami,winston hills,4223,nsw,19151111,5304218
rec-1016-org,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625
rec-4405-org,charles,green,38,salkauskas crescent,kela,dapto,4566,nsw,19480930,4365168
rec-1288-org,vanessa,parr,905,macquoid place,broadbridge manor,south grafton,2135,sa,19951119,9239102
rec-3585-org,mikayla,malloney,37,randwick road,avalind,hoppers crossing,4552,vic,19860208,7207688
...,...,...,...,...,...,...,...,...,...,...
rec-2153-org,annabel,grierson,97,mclachlan crescent,lantana lodge,broome,2480,nsw,19840224,7676186
rec-1604-org,sienna,musolino,22,smeaton circuit,pangani,mckinnon,2700,nsw,19890525,4971506
rec-1003-org,bradley,matthews,2,jondol place,horseshoe ck,jacobs well,7018,sa,19481122,8927667


In [3]:
print(f'Length of dataframe dfA: {len(dfA)}')

Length of dataframe dfA: 5000


In [4]:
dfB

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-561-dup-0,elton,NaN,3,light setreet,pinehill,windermere,3212,vic,19651013,1551941
rec-2642-dup-0,mitchell,maxon,47,edkins street,lochaoair,north ryde,3355,nsw,19390212,8859999
rec-608-dup-0,NaN,white,72,lambrigg street,kelgoola,broadbeach waters,3159,vic,19620216,9731855
rec-3239-dup-0,elk i,menzies,1,lyster place,NaN,northwood,2585,vic,19980624,4970481
rec-2886-dup-0,NaN,garanggar,NaN,may maxwell crescent,springettst arcade,forest hill,2342,vic,19921016,1366884
...,...,...,...,...,...,...,...,...,...,...
rec-4495-dup-0,connor,belperio,15,NaN,NaN,ryde,2570,nsw,19170518,5394641
rec-4211-dup-0,daniel,maspn,9,derrington crescent,el pedro caravan park,sunnybank,4350,vic,19500705,5525378
rec-3131-dup-0,samuel,crofs,613,banjine street,kurrajong vlge,pengzin,2230,qld,19410531,4467228


In [5]:
print(f'Length of dataframe dfB: {len(dfB)}')

Length of dataframe dfB: 5000


In [6]:
miTrueLinks

MultiIndex([(   'rec-0-org',    'rec-0-dup-0'),
            (   'rec-1-org',    'rec-1-dup-0'),
            (   'rec-2-org',    'rec-2-dup-0'),
            (   'rec-3-org',    'rec-3-dup-0'),
            (   'rec-4-org',    'rec-4-dup-0'),
            (   'rec-5-org',    'rec-5-dup-0'),
            (   'rec-6-org',    'rec-6-dup-0'),
            (   'rec-7-org',    'rec-7-dup-0'),
            (   'rec-8-org',    'rec-8-dup-0'),
            (   'rec-9-org',    'rec-9-dup-0'),
            ...
            ('rec-4990-org', 'rec-4990-dup-0'),
            ('rec-4991-org', 'rec-4991-dup-0'),
            ('rec-4992-org', 'rec-4992-dup-0'),
            ('rec-4993-org', 'rec-4993-dup-0'),
            ('rec-4994-org', 'rec-4994-dup-0'),
            ('rec-4995-org', 'rec-4995-dup-0'),
            ('rec-4996-org', 'rec-4996-dup-0'),
            ('rec-4997-org', 'rec-4997-dup-0'),
            ('rec-4998-org', 'rec-4998-dup-0'),
            ('rec-4999-org', 'rec-4999-dup-0')],
           length=5000)

### Preprocessing
Because we are using a synthetic dataset, the amount of scrubbing we need to do is minor relative to real-wold use cases (as we'll see later). In fact, much of the scrubbing we'll do on this dataset is done to align it with our real-world data. However, along with scrubbing we need to enable efficient comparison of data. This means that we want to be able to do phonetic comparisons of strings rather than relying on exact matching, strongly type dates for more efficient comparison, etc. As with most data science efforts, determining optimal preprocessing is usually a trial-and-error process.

In [7]:
# preprocess fields: given_name, surname, street_number, address_1, address_2, suburb, state, postcode, soc_sec_id, date_of_birth

# force all strings to upper case
dfA = dfA.applymap(lambda x: x.upper() if isinstance(x, str) else x)
dfB = dfB.applymap(lambda x: x.upper() if isinstance(x, str) else x)

# convert city abbreviations to names
def convert_au_state(short_name):
    if short_name == 'ACT':
        return 'AUSTRALIAN CAPITAL TERRITORY'
    elif short_name == 'NSW':
        return 'NEW SOUTH WALES'
    elif short_name == 'NT':
        return 'NORTHERN TERRITORY'
    elif short_name == 'QLD':
        return 'QUEENSLAND'
    elif short_name == 'SA':
        return 'SOUTH AUSTRALIA'
    elif short_name == 'TAS':
        return 'TANZANIA'
    elif short_name == 'VIC':
        return 'VICTORIA'
    else:
        return short_name
dfA['state'] = dfA['state'].apply(convert_au_state)
dfB['state'] = dfB['state'].apply(convert_au_state)

# process name fields phonetically
dfA[['given_name', 'surname']] = dfA[['given_name', 'surname']].fillna('')
dfB[['given_name', 'surname']] = dfB[['given_name', 'surname']].fillna('')
dfA['given_name_ph'] = phonetic(dfA['given_name'],method='metaphone',decode_error='replace')
dfB['given_name_ph'] = phonetic(dfB['given_name'],method='metaphone',decode_error='replace')
dfA['surname_ph'] = phonetic(dfA['surname'],method='metaphone',decode_error='replace')
dfB['surname_ph'] = phonetic(dfB['surname'],method='metaphone',decode_error='replace')

# create a full name field
dfA['full_name'] = dfA['given_name'] + ' ' + dfA['surname']
dfA['full_name'] = dfA['full_name'].str.strip()
dfB['full_name'] = dfB['given_name'] + ' ' + dfB['surname']
dfB['full_name'] = dfB['full_name'].str.strip()

# process suburb phonetically
dfA[['suburb', 'state']] = dfA[['suburb', 'state']].fillna('')
dfB[['suburb', 'state']] = dfB[['suburb', 'state']].fillna('')
dfA['suburb_ph'] = phonetic(dfA['suburb'],method='metaphone',decode_error='replace')
dfB['suburb_ph'] = phonetic(dfA['suburb'],method='metaphone',decode_error='replace')

# replace nans in postcode and convert to int
dfA['postcode'] = dfA['postcode'].fillna('0')
dfB['postcode'] = dfB['postcode'].fillna('0')
dfA['postcode_int'] = pd.to_numeric(dfA['postcode'])
dfB['postcode_int'] = pd.to_numeric(dfB['postcode'])

# replace nans in soc_sec_id and convert to int
dfA['soc_sec_id'] = dfA['soc_sec_id'].fillna('0')
dfB['soc_sec_id'] = dfB['soc_sec_id'].fillna('0')
dfA['soc_sec_id_int'] = dfA['soc_sec_id'].apply(int)
dfB['soc_sec_id_int'] = dfB['soc_sec_id'].apply(int)

# cast date_of_birth as datetime64 in a new column
dfA['dob_typed'] = pd.to_datetime(dfA['date_of_birth'],format='%Y%m%d',errors='coerce')
dfB['dob_typed'] = pd.to_datetime(dfB['date_of_birth'],format='%Y%m%d',errors='coerce')

# build an all-text column
dfA['text'] = dfA['full_name'] + ' ' + dfA['suburb'] + ' ' + dfA[ 'state']
dfB['text'] = dfB['full_name'] + ' ' + dfB['suburb'] + ' ' + dfB[ 'state']

dfA


,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,given_name_ph,surname_ph,full_name,suburb_ph,postcode_int,soc_sec_id_int,dob_typed,text
rec_id,,,,,,,,,,,,,,,,,,
rec-1070-org,MICHAELA,NEUMANN,8,STANLEY STREET,MIAMI,WINSTON HILLS,4223,NEW SOUTH WALES,19151111,5304218,MXL,NMN,MICHAELA NEUMANN,WNSTNHLS,4223,5304218,1915-11-11,MICHAELA NEUMANN WINSTON HILLS NEW SOUTH WALES
rec-1016-org,COURTNEY,PAINTER,12,PINKERTON CIRCUIT,BEGA FLATS,RICHLANDS,4560,VICTORIA,19161214,4066625,KRTN,PNTR,COURTNEY PAINTER,RXLNTS,4560,4066625,1916-12-14,COURTNEY PAINTER RICHLANDS VICTORIA
rec-4405-org,CHARLES,GREEN,38,SALKAUSKAS CRESCENT,KELA,DAPTO,4566,NEW SOUTH WALES,19480930,4365168,XRLS,KRN,CHARLES GREEN,TPT,4566,4365168,1948-09-30,CHARLES GREEN DAPTO NEW SOUTH WALES
rec-1288-org,VANESSA,PARR,905,MACQUOID PLACE,BROADBRIDGE MANOR,SOUTH GRAFTON,2135,SOUTH AUSTRALIA,19951119,9239102,FNS,PR,VANESSA PARR,S0KRFTN,2135,9239102,1995-11-19,VANESSA PARR SOUTH GRAFTON SOUTH AUSTRALIA
rec-3585-org,MIKAYLA,MALLONEY,37,RANDWICK ROAD,AVALIND,HOPPERS CROSSING,4552,VICTORIA,19860208,7207688,MKL,MLN,MIKAYLA MALLONEY,HPRSKRSNK,4552,7207688,1986-02-08,MIKAYLA MALLONEY HOPPERS CROSSING VICTORIA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
rec-2153-org,ANNABEL,GRIERSON,97,MCLACHLAN CRESCENT,LANTANA LODGE,BROOME,2480,NEW SOUTH WALES,19840224,7676186,ANBL,KRRSN,ANNABEL GRIERSON,BRM,2480,7676186,1984-02-24,ANNABEL GRIERSON BROOME NEW SOUTH WALES
rec-1604-org,SIENNA,MUSOLINO,22,SMEATON CIRCUIT,PANGANI,MCKINNON,2700,NEW SOUTH WALES,19890525,4971506,SN,MSLN,SIENNA MUSOLINO,MKNN,2700,4971506,1989-05-25,SIENNA MUSOLINO MCKINNON NEW SOUTH WALES
rec-1003-org,BRADLEY,MATTHEWS,2,JONDOL PLACE,HORSESHOE CK,JACOBS WELL,7018,SOUTH AUSTRALIA,19481122,8927667,BRTL,M0S,BRADLEY MATTHEWS,JKBSWL,7018,8927667,1948-11-22,BRADLEY MATTHEWS JACOBS WELL SOUTH AUSTRALIA


### Indexing
The next step in the process is to link each row in the source to each row in the target so that we can check for matches based on weighted scores. In it's simplest form, this results in a full cartesian product of `dfA X dfB`. Clearly this does not scale well so most often a technique called Blocking is applied in which obvious non-matches are not included in the indexes. Simpler techniques block based on exact matches on one or more attributes (columns) while more complex and interesting techniques relax the matching criteria using techniques such as clustering, fuzzy hashing, and bloom filter grouping.

The PRLT library includes fairly simple Indexing methods that include Full (no Blocking), Block which uses exact matches on specified attributes, and SortedNeighborhood which sorts data from both datasets together and groups data from two different sets according a window size (<ins>de Bruin, 2023</ins>). Interestingly, this concept is similar in functionality to convolution.

In [8]:
# full indexing --> far too many rows to process
indexer = recordlinkage.Index()
indexer.full()
pairs = indexer.index(dfA, dfB)
pairs

MultiIndex([('rec-1070-org',  'rec-561-dup-0'),
            ('rec-1070-org', 'rec-2642-dup-0'),
            ('rec-1070-org',  'rec-608-dup-0'),
            ('rec-1070-org', 'rec-3239-dup-0'),
            ('rec-1070-org', 'rec-2886-dup-0'),
            ('rec-1070-org', 'rec-4285-dup-0'),
            ('rec-1070-org',  'rec-929-dup-0'),
            ('rec-1070-org', 'rec-4833-dup-0'),
            ('rec-1070-org',  'rec-717-dup-0'),
            ('rec-1070-org', 'rec-3984-dup-0'),
            ...
            (  'rec-66-org',  'rec-670-dup-0'),
            (  'rec-66-org', 'rec-4134-dup-0'),
            (  'rec-66-org', 'rec-3866-dup-0'),
            (  'rec-66-org', 'rec-3152-dup-0'),
            (  'rec-66-org', 'rec-3363-dup-0'),
            (  'rec-66-org', 'rec-4495-dup-0'),
            (  'rec-66-org', 'rec-4211-dup-0'),
            (  'rec-66-org', 'rec-3131-dup-0'),
            (  'rec-66-org', 'rec-3815-dup-0'),
            (  'rec-66-org',  'rec-493-dup-0')],
           names=['rec_

In [9]:
# exact match on phonetic surname --> not ideal based on the types of errors we expect to encounter
indexer = recordlinkage.Index()
indexer.block(left_on='surname_ph', right_on='surname_ph')
pairs = indexer.index(dfA, dfB)
pairs

MultiIndex([('rec-1070-org', 'rec-2672-dup-0'),
            ('rec-1070-org', 'rec-4387-dup-0'),
            ('rec-1070-org',  'rec-787-dup-0'),
            ('rec-1070-org', 'rec-2158-dup-0'),
            ('rec-1016-org', 'rec-3267-dup-0'),
            ('rec-1016-org', 'rec-2136-dup-0'),
            ('rec-1016-org', 'rec-1948-dup-0'),
            ('rec-1016-org', 'rec-1016-dup-0'),
            ('rec-1016-org', 'rec-1032-dup-0'),
            ('rec-1016-org', 'rec-1321-dup-0'),
            ...
            ('rec-1003-org', 'rec-3462-dup-0'),
            ('rec-1003-org', 'rec-1014-dup-0'),
            ('rec-4883-org', 'rec-1662-dup-0'),
            ('rec-4883-org',  'rec-835-dup-0'),
            ('rec-4883-org', 'rec-4883-dup-0'),
            ('rec-4883-org', 'rec-1884-dup-0'),
            ('rec-4883-org',  'rec-785-dup-0'),
            ('rec-4883-org', 'rec-3923-dup-0'),
            ('rec-4883-org', 'rec-2609-dup-0'),
            ('rec-4883-org', 'rec-4459-dup-0')],
           names=['rec_

#### Custom LSH Indexing Algorithm

The PRLT library also allows the Index class to be extended to enable other blocking algorithms. One newer blocking algorithm found in the literature uses Locality Sensitive Hashing (LSH). The LSH algorithm has gained popularity for two reasons. First, it is less strict than standard blocking but still effectively reduces the comparison size relative to full indexing. Second, because the algorithm hashes values before indexing using LSH, it can be used to preserve the privacy of the original data sets (<ins>Dutt, 2023</ins>). In the code below, we extend the PRLT BaseIndexAlgorithm class using the [datasketch library](https://ekzhu.com/datasketch/index.html)'s MinHash and MinHashLSH algorithms. MinHash estimates the Jaccard Similarity Index for two inputs. These are then loaded into the MinHashed LSH index and matches are queried (<ins>Zhu, 2024</ins>). The resulting matches are output as Pandas MultiArray so that they work seamlessly within the PRLT libary. Note that two tokenizers are enabled. The first tokenizes text bigrams and the second tokenizes words. Both will be experimented with.

In order to optimize performance, the following techniques were applied to the code below:

1. Static tokenizer methods were moved out of the class so that they would be precompiled
2. All MinHash and LSH processing was done in-place using dataframes
3. The [swifter library](https://github.com/jmcarpenter2/swifter/blob/master/docs/documentation.md) was used to parallelize the dataframe.apply method for generating MinHashes


In [10]:
def _tokenize_bigram(string):
    # Clean and create character bigrams
    string = re.sub(r'\W+', '', string.lower())
    tokens = set(string[i:i+2] for i in range(len(string)-1))
    return tokens

def _tokenize_words(string):
    # Clean and create character bigrams
    tokens = string.split()
    return tokens

class LSHIndex(BaseIndexAlgorithm):

    def __init__(self, column, threshold=0.3, num_perm=128, tokenizer='words'):
        super().__init__()
        self.column = column
        self.threshold = threshold
        self.num_perm = num_perm
        self.tokenizer = tokenizer

    def _tokenize(self, string):
        if self.tokenizer == 'words':
            return _tokenize_words(string)
        elif self.tokenizer == 'bigrams':
            return _tokenize_bigram(string)

    def _create_minhash(self, row):
        string = row[self.column]
        tokens = self._tokenize(string)
        m = MinHash(num_perm=self.num_perm)
        for token in tokens:
            m.update(token.encode('utf8'))
        return LeanMinHash(m)

    def _link_index(self, dfA, dfB):
        is_deduplication = dfA.equals(dfB)

        # MinHash computation
        start = time.perf_counter()
        #dfA['minhash'] = dfA.apply(func=self._create_minhash, axis=1)
        dfA['minhash'] = dfA.swifter.apply(func=self._create_minhash, axis=1)
        end = time.perf_counter()
        print(f'Seconds to compute minhash dfA: {end - start}')
        if is_deduplication:
            dfB = dfA
        else:
            start = time.perf_counter()
            dfB['minhash'] = dfB.swifter.apply(func=self._create_minhash, axis=1)
            end = time.perf_counter()
            print(f'Seconds to compute minhash dfB: {end - start}')

        # Insert into LSH
        lsh = MinHashLSH(threshold=self.threshold, num_perm=self.num_perm)
        start = time.perf_counter()
        for index, row in dfB.iterrows():
            lsh.insert(index, row['minhash'])
        end = time.perf_counter()
        print(f'Seconds to build LSH of dfB: {end - start}')

        # Query LSH for candidate pairs
        l_rows = []
        start = time.perf_counter()
        for index, row in dfA.iterrows():
            result = lsh.query(row['minhash'])
            for idx_b in result:
                if is_deduplication and index >= idx_b:
                    continue
                l_rows.append({'dfA_idx': index, 'dfB_idx': idx_b})
        df_out = pd.DataFrame(l_rows)
        end = time.perf_counter()
        print(f'Seconds to query dfB for each row of dfA: {end - start}')

        # Convert result to MultiIndex
        start = time.perf_counter()
        output = pd.MultiIndex.from_frame(df_out)
        end = time.perf_counter()
        print(f'Seconds to convert df to MultiIndex: {end - start}')
        return output

#### Indexing Results Using LSHIndex

Below we index our dataset using the custom LSHIndex Blocking algorithm. Two different tokenization methods were attempted - a `bigram` tokenizer to compare the `full_name` column and a word-based tokenizer to tokenize the `text` column. While the word tokenizer is more sensitive, the bigram tokenizer yielded better results in the comparison tests performed later. We found that the default settings for Jaccard Similarity threshold and number of MinHash permutations worked well with bigram, but needed to be adjusted for word comparison. More information on this can be found in the DataSketch documentation (<ins>Zhu, 2024</ins>) and in a reference implementation by Martin Boyanov (<ins>Boyanov, 2020</ins>).

In [11]:
indexer = LSHIndex(column='full_name', threshold=0.5, num_perm=128, tokenizer='bigrams')
pairs_lsh = indexer.index(dfA, dfB)
pairs_lsh

Pandas Apply: 100%|██████████| 5000/5000 [00:05<00:00, 934.22it/s]


Seconds to compute minhash dfA: 6.009883600054309


Pandas Apply: 100%|██████████| 5000/5000 [00:05<00:00, 944.79it/s]


Seconds to compute minhash dfB: 5.930727800005116
Seconds to build LSH of dfB: 0.5911461000796407
Seconds to query dfB for each row of dfA: 0.3874913000036031
Seconds to convert df to MultiIndex: 0.009221100015565753


MultiIndex([('rec-1070-org', 'rec-2754-dup-0'),
            ('rec-1070-org', 'rec-2797-dup-0'),
            ('rec-1016-org', 'rec-1948-dup-0'),
            ('rec-1016-org',  'rec-865-dup-0'),
            ('rec-1016-org', 'rec-3267-dup-0'),
            ('rec-1016-org', 'rec-1151-dup-0'),
            ('rec-1016-org', 'rec-1321-dup-0'),
            ('rec-1016-org',  'rec-355-dup-0'),
            ('rec-1016-org', 'rec-1016-dup-0'),
            ('rec-4405-org', 'rec-2078-dup-0'),
            ...
            ('rec-1003-org', 'rec-3078-dup-0'),
            ('rec-1003-org', 'rec-3226-dup-0'),
            ('rec-1003-org', 'rec-1771-dup-0'),
            ('rec-1003-org',  'rec-942-dup-0'),
            ('rec-1003-org', 'rec-4118-dup-0'),
            ('rec-1003-org', 'rec-4433-dup-0'),
            ('rec-4883-org', 'rec-1194-dup-0'),
            ('rec-4883-org', 'rec-4883-dup-0'),
            ('rec-4883-org', 'rec-1099-dup-0'),
            (  'rec-66-org',   'rec-66-dup-0')],
           names=['rec_

#### Index Runtime Benchmarks:

Time to generate the Pandas MultiIndex for two 5000 row datasets (see Appendix 1 for system info):

- *Full Index Generation*: 118ms / 25M rows
- *Exact-Match Blocking Index Generation*: 25ms / 89727 rows
- *LSH Blocking Index Generation*: 13.5s / 507726 rows


### Comparison

When comparing the candidate records generated by the Indexing steps above in order to generate comparison scores, there are a number of algorithms that can be applied. The PRLT library includes several methods that compare data based on it's type. These comparisons, run alone, all return scores. When a Pandas dataframe is provided with a set of records, the library can be used to compute scores for each row in the dataframe. Each matching column between the two datasets can be assigned it's own comparison algorithm. The result in this case is a vector that includes the row keys of the two rows being compared and columns for each compared variable that includes their match score.

#### Algorithms by Data Type

##### General

  - Exact --> straight equivalence that returns a score of 1 or 0
  - Custom --> As with Indexers above, the library offers the BaseCompareFeature from which custom algorithms can be implemented

##### String

String comparisons are implemented in the PRLT library by wrapping the Python [jellyfish](https://jamesturk.github.io/jellyfish/) library for approximate and phonetic string matching. PRLT implements the following string comparison methods from the jellyfish library:

   - Jaro
   - Jaro-Winkler
   - Levenshtein
   - Damerau-Levenshtein

Interestingly, it excludes Hamming Distance and Match Rating Approach algorithms. The PRLT library also excludes the phonetic encoding tools provided in the jellyfish library. These tools are interesting in that they address phonetic misspellings that might fail edit distance algorithms like the ones above. For example, the spellings Clumps and Klumpz sound the same but would have a low edit distance score. Applying a phonetic encoding tool early can improve match scores (<ins>Zhu, 2024</ins>).

##### Numeric

The PRLT library can compare numeric values using a variety of nearness measures all of which are based on a distance from a comparison point. The following diagram, referenced from the PRLT documentation, well illustrates how the comparisons work (<ins>Elastic, 2025</ins>):

<br><center>
![numeric matching algorithms](images/elas_1705.png)
</center>

As the diagram shows, two values are compared as a distance using one of several decay algorithms (step, linear, exponential, gaussian or squared), returning a float value between 0 and 1.

##### Geospatial

For locations that include lattidue and longitude, the toolkit can compute the haversine distance between coordinates and then compute the numeric similarity between the distances as described for numeric values above.

##### Date

Again, the numeric distance between dates is computed with a few date-specific scoring optimizations applied.

- score when month and day are swapped: allows for a set score to be applied if the only difference between two dates is that day and month are transposed
- score for common month swapping errors: allows for a set score to be applied if the only difference between two dates is that a common string to numeric date conversion error occurred

##### Precomputed Variable

In addition to row-specific values, the toolkit allows for precomputed scores already contained in the dataset to be passed in as variables. By default, these variables are normalized to a score range from 0-1.

#### Comparison Algorithm Applied by Column

- *given_name_ph*: exact
- *surname_ph*: exact
- *suburb_ph*: exact
- *postcode*: numeric (exponential)
- *state*: Jaro-Winkler distance
- *date_of_birth*: date with default scoring for month/day and text to numeric month errors
- *soc_sec_id*: numeric (exponential)

In [12]:
comparison = recordlinkage.Compare()
comparison.exact(left_on="given_name_ph", right_on="given_name_ph", label="given_name_ph")
comparison.exact(left_on="surname_ph", right_on="surname_ph", label="surname_ph")
comparison.string(left_on="given_name", right_on="given_name", method='jarowinkler', label='given_name')
comparison.string(left_on="surname", right_on="state", method='jarowinkler', label='surname')
# comparison.string(left_on="street_number", right_on="street_number", method='damerau_levenshtein', label='street_number')
# comparison.string(left_on="address_1", right_on="address_1", method='damerau_levenshtein', label='address_1')
# comparison.string(left_on="address_2", right_on="address_2", method='damerau_levenshtein', label='address_2')
comparison.exact(left_on="suburb_ph", right_on="suburb_ph", label='suburb_ph')
comparison.string(left_on="suburb", right_on="suburb", label='suburb')
comparison.numeric (left_on="postcode_int", right_on="postcode_int", method='exp', label='postcode')
comparison.exact(left_on="state", right_on="state", label='state')
comparison.numeric(left_on="soc_sec_id_int", right_on="soc_sec_id_int", method="exp", label='soc_sec_id')
comparison.date(left_on="dob_typed", right_on="dob_typed", label='dob')
print(len(pairs))
features = comparison.compute(pairs=pairs, x=dfA, x_link=dfB)
features

105304


U:\Users\v_mar\OneDrive\Documents\KU\CPSC548\FinalProject\record_linkage\venv\Lib\site-packages\recordlinkage\algorithms\string.py:56: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - levenshtein_distance(x[0], x[1]) / np.max([len(x[0]), len(x[1])])


given_name_ph  surname_ph  given_name   surname  \
rec_id_1     rec_id_2                                                          
rec-1070-org rec-2672-dup-0              0           1    0.638889  0.000000   
             rec-4387-dup-0              0           1    0.550000  0.671429   
             rec-787-dup-0               0           1    0.430556  0.422619   
             rec-2158-dup-0              0           1    0.441667  0.671429   
rec-1016-org rec-3267-dup-0              0           1    0.000000  0.431746   
...                                    ...         ...         ...       ...   
rec-4883-org rec-1884-dup-0              0           1    0.444444  0.377778   
             rec-785-dup-0               0           1    0.000000  0.438889   
             rec-3923-dup-0              0           1    0.000000  0.000000   
             rec-2609-dup-0              0           1    0.844444  0.000000   
             rec-4459-dup-0              0           1    0.577778  0.377778   

                             suburb_ph    suburb       postcode  state  \
rec_id_1     rec_id_2                                                    
rec-1070-org rec-2672-dup-0          0  0.307692   0.000000e+00      0   
             rec-4387-dup-0          0  0.000000   0.000000e+00      0   
             rec-787-dup-0           0  0.076923   0.000000e+00      0   
             rec-2158-dup-0          0  0.307692  1.430222e-247      0   
rec-1016-org rec-3267-dup-0          0  0.222222   0.000000e+00      0   
...                                ...       ...            ...    ...   
rec-4883-org rec-1884-dup-0          0  0.125000  3.131513e-294      0   
             rec-785-dup-0           0  0.111111   0.000000e+00      0   
             rec-3923-dup-0          0  0.111111   7.812500e-03      0   
             rec-2609-dup-0          0  0.222222   0.000000e+00      0   
             rec-4459-dup-0          0  0.181818   0.000000e+00      0   

                             soc_sec_id  dob  
rec_id_1     rec_id_2                         
rec-1070-org rec-2672-dup-0         0.0  0.0  
             rec-4387-dup-0         0.0  0.0  
             rec-787-dup-0          0.0  0.0  
             rec-2158-dup-0         0.0  0.0  
rec-1016-org rec-3267-dup-0         0.0  0.0  
...                                 ...  ...  
rec-4883-org rec-1884-dup-0         0.0  0.0  
             rec-785-dup-0          0.0  0.0  
             rec-3923-dup-0         0.0  0.0  
             rec-2609-dup-0         0.0  0.0  
             rec-4459-dup-0         0.0  0.0  

[105304 rows x 10 columns]

In [13]:
features.describe()

,given_name_ph,surname_ph,given_name,surname,suburb_ph,suburb,postcode,state,soc_sec_id,dob
count,105304.000000,105304.0,105304.000000,105304.000000,105304.0,105304.000000,1.053040e+05,105304.000000,105304.000000,105304.000000
mean,0.033361,1.0,0.401377,0.379457,0.0,0.164120,3.214702e-02,0.245252,0.031903,0.031542
std,0.179577,0.0,0.258514,0.192889,0.0,0.181727,1.738743e-01,0.430238,0.175736,0.174771
min,0.000000,1.0,0.000000,0.000000,0.0,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000,1.0,0.000000,0.344444,0.0,0.076923,0.000000e+00,0.000000,0.000000,0.000000
50%,0.000000,1.0,0.464286,0.438889,0.0,0.125000,0.000000e+00,0.000000,0.000000,0.000000
75%,0.000000,1.0,0.555556,0.488889,0.0,0.200000,2.002083e-146,0.000000,0.000000,0.000000
max,1.000000,1.0,1.000000,0.822222,0.0,1.000000,1.000000e+00,1.000000,1.000000,1.000000


In [14]:
features_lsh = comparison.compute(pairs=pairs_lsh, x=dfA, x_link=dfB)
features_lsh

U:\Users\v_mar\OneDrive\Documents\KU\CPSC548\FinalProject\record_linkage\venv\Lib\site-packages\recordlinkage\algorithms\string.py:56: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - levenshtein_distance(x[0], x[1]) / np.max([len(x[0]), len(x[1])])


given_name_ph  surname_ph  given_name   surname  \
rec_id_1     rec_id_2                                                          
rec-1070-org rec-2754-dup-0              0           0    0.441667  0.671429   
             rec-2797-dup-0              0           0    0.511905  0.671429   
rec-1016-org rec-1948-dup-0              0           1    0.441667  0.601190   
             rec-865-dup-0               0           0    0.550000  0.328571   
             rec-3267-dup-0              0           1    0.000000  0.431746   
...                                    ...         ...         ...       ...   
rec-1003-org rec-4433-dup-0              0           1    0.447619  0.416667   
rec-4883-org rec-1194-dup-0              0           0    0.577778  0.566667   
             rec-4883-dup-0              1           1    1.000000  0.566667   
             rec-1099-dup-0              0           0    0.472222  0.377778   
rec-66-org   rec-66-dup-0                1           0    1.000000  0.403704   

                             suburb_ph    suburb       postcode  state  \
rec_id_1     rec_id_2                                                    
rec-1070-org rec-2754-dup-0          0  0.153846   1.274474e-57      0   
             rec-2797-dup-0          0  0.153846   0.000000e+00      0   
rec-1016-org rec-1948-dup-0          0  0.111111   0.000000e+00      1   
             rec-865-dup-0           0  0.153846   6.681912e-52      0   
             rec-3267-dup-0          0  0.222222   0.000000e+00      0   
...                                ...       ...            ...    ...   
rec-1003-org rec-4433-dup-0          0  0.181818   0.000000e+00      0   
rec-4883-org rec-1194-dup-0          0  0.000000   0.000000e+00      1   
             rec-4883-dup-0          0  1.000000   1.000000e+00      1   
             rec-1099-dup-0          0  0.111111  5.180654e-318      0   
rec-66-org   rec-66-dup-0            0  1.000000   1.000000e+00      1   

                             soc_sec_id  dob  
rec_id_1     rec_id_2                         
rec-1070-org rec-2754-dup-0         0.0  0.0  
             rec-2797-dup-0         0.0  0.0  
rec-1016-org rec-1948-dup-0         0.0  0.0  
             rec-865-dup-0          0.0  0.0  
             rec-3267-dup-0         0.0  0.0  
...                                 ...  ...  
rec-1003-org rec-4433-dup-0         0.0  0.0  
rec-4883-org rec-1194-dup-0         0.0  0.0  
             rec-4883-dup-0         1.0  1.0  
             rec-1099-dup-0         0.0  0.0  
rec-66-org   rec-66-dup-0           1.0  1.0  

[38048 rows x 10 columns]

In [15]:
features_lsh.describe()

,given_name_ph,surname_ph,given_name,surname,suburb_ph,suburb,postcode,state,soc_sec_id,dob
count,38048.000000,38048.000000,38048.000000,38048.000000,38048.0,38048.000000,3.804800e+04,38048.000000,38048.000000,38048.000000
mean,0.356760,0.442967,0.621522,0.397308,0.0,0.222854,9.551145e-02,0.296783,0.099974,0.098363
std,0.479049,0.496743,0.356339,0.179041,0.0,0.272287,2.919576e-01,0.456846,0.299948,0.297797
min,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.447619,0.408333,0.0,0.083333,0.000000e+00,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.611111,0.441667,0.0,0.142857,4.144523e-317,0.000000,0.000000,0.000000
75%,1.000000,1.000000,1.000000,0.505556,0.0,0.230769,2.996273e-95,1.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,0.866667,0.0,1.000000,1.000000e+00,1.000000,1.000000,1.000000


#### Comparison Runtime Benchmarks:

Time to run a comparison of two 5000 row datasets (see Appendix 1 for system info):

- *Full Index*: 27m, 18s
- *Exact-Match Blocking Index*: 5.5s
- *LSH Blocking Index*: 25.1s



## Classification

The PRLT library includes the following classifiers that can be used to classify matches based on the features generated above:

Supervised:

- Logistic Regression Classifier
- Naive Bayesian Classifier
- Support Vector Machine (SVM) Classifier

Unsupervised:

- Expectation/Conditional Maximization (ECM) Classifier
- KMeans Classifier

### Unsupervised Classifiers

In [16]:
# k-means clustering using exact match blocking
kmeans = recordlinkage.KMeansClassifier()
result_kmeans = kmeans.fit_predict(features)
recordlinkage.confusion_matrix(links_true=miTrueLinks, links_pred=result_kmeans, total=len(dfA.index))

array([[3693, 1307],
       [   1,   -1]])

In [17]:
recordlinkage.fscore(links_true=miTrueLinks, links_pred=result_kmeans)

0.849551414768806

In [18]:
# k-means clustering using LSH blocking
kmeans = recordlinkage.KMeansClassifier()
result_kmeans = kmeans.fit_predict(features_lsh)
recordlinkage.confusion_matrix(links_true=miTrueLinks, links_pred=result_kmeans, total=len(dfA.index))

array([[4156,  844],
       [   0,    0]])

In [19]:
recordlinkage.fscore(links_true=miTrueLinks, links_pred=result_kmeans)

0.9078200087374401

In [20]:
# ECM using exact blocking
ecm = recordlinkage.ECMClassifier(binarize=0.8)
result_ecm = ecm.fit_predict(features)
recordlinkage.confusion_matrix(links_true=miTrueLinks, links_pred=result_ecm, total=len(dfA.index))

array([[3712, 1288],
       [   5,   -5]])

In [21]:
recordlinkage.fscore(links_true=miTrueLinks, links_pred=result_ecm)

0.8516691522312723

In [22]:
# ECM using LSH blocking
ecm = recordlinkage.ECMClassifier(binarize=0.8)
result_ecm = ecm.fit_predict(features_lsh)
recordlinkage.confusion_matrix(links_true=miTrueLinks, links_pred=result_ecm, total=len(dfA.index))

array([[4171,  829],
       [   4,   -4]])

In [23]:
recordlinkage.fscore(links_true=miTrueLinks, links_pred=result_ecm)

0.9092098092643053

The results above demonstrate that LSH provides better results than exact-match blocking when dealing with dirty data in the blocking set. However, classification performance is not great.

### Supervised Classifiers


In [24]:
# create a training set by splitting data
training_dfA = dfA[0:500]
training_dfB = dfB[0:500]
training_match_idx = miTrueLinks[miTrueLinks.get_level_values(0).isin(training_dfA.index)]

# index training set using LSH
training_pairs_lsh = indexer.index(training_dfA, training_dfB)

# compare training data
training_features_lsh = comparison.compute(pairs=training_pairs_lsh, x=training_dfA, x_link=training_dfB)

#classify using Naive Bayes Classifier
nbc = recordlinkage.NaiveBayesClassifier(binarize=0.8)
nbc.fit(training_features_lsh, training_match_idx)
result_nbc = nbc.predict(features_lsh)
recordlinkage.confusion_matrix(links_true=miTrueLinks, links_pred=result_nbc, total=len(dfA.index))


Pandas Apply: 100%|██████████| 500/500 [00:00<00:00, 931.29it/s]
C:\Users\v_mar\AppData\Local\Temp\ipykernel_49300\246946091.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfA['minhash'] = dfA.swifter.apply(func=self._create_minhash, axis=1)


Seconds to compute minhash dfA: 0.6090634000720456


Pandas Apply: 100%|██████████| 500/500 [00:00<00:00, 919.03it/s]
C:\Users\v_mar\AppData\Local\Temp\ipykernel_49300\246946091.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfB['minhash'] = dfB.swifter.apply(func=self._create_minhash, axis=1)


Seconds to compute minhash dfB: 0.6167102999752387
Seconds to build LSH of dfB: 0.04140750004444271
Seconds to query dfB for each row of dfA: 0.037738599930889904
Seconds to convert df to MultiIndex: 0.000710100051946938


array([[4174,  826],
       [  38,  -38]])

In [25]:
recordlinkage.fscore(links_true=miTrueLinks, links_pred=result_nbc)

0.9062092922275292

In [26]:
recordlinkage.precision(links_true=miTrueLinks, links_pred=result_nbc)

0.9909781576448243

In [27]:
recordlinkage.recall(links_true=miTrueLinks, links_pred=result_nbc)

0.8348

In [28]:
#classify using SVM
svc = recordlinkage.SVMClassifier() # options --> ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’
svc.fit(training_features_lsh, training_match_idx)
result_svc = svc.predict(features_lsh)
recordlinkage.confusion_matrix(links_true=miTrueLinks, links_pred=result_svc, total=len(dfA.index))

array([[4160,  840],
       [   0,    0]])

In [29]:
recordlinkage.fscore(links_true=miTrueLinks, links_pred=result_svc)

0.9082969432314411

## Real World Case

Identifying personnel involved in clinical trials is a real-world challenge. For companies that sell software to organizations that perform clinical trials, having consistent and correct information about the physicians involved in clinical trials provides real value to customers. Often the same investigators participate in many clinical trials across many trial sponsors, each of whom have their own record of the investigator. Matching and deduplicating these records represents a challenge that all vendors have.

The United States government requires providers that participate in electronic transactions specified in HIPAA to register with the National Plan & Provider Enumeration System (NPPES). When they register, they are supplied with National Provider Identifier (NPI) number. The NPPES database is available for download as text files.

The government also requires that clinical trials run in the United States be registered online at the ClinicalTrials.gov website. This registry contains information about trials once they are registered with the FDA. The database includes information about investigators that are currently enrolling patients in clinical trials. Snapshots of the database are available for download.

Using the PRLT library, we will attempt to match investigators between these two databases.

The ClinicalTrials.gov investigators list was generated by downloading a snapshot of a PostgreSQL database from the [Clinical Trials Transformation Initiative](https://aact.ctti-clinicaltrials.org/download)'s website. The snapshot downloaded was from 2025-04-30. Once downloaded, the database was restored to a local PostgreSQL 17 instance and was analyzed. The following query was used to generate a csv file used for this analysis.

    select
        b.name as full_name,
        a.city as city,
        a.state as state,
        a.zip as postal_code,
        a.country as country,
        a.nct_id
    from
        ctgov.facilities a
    inner join
        ctgov.facility_investigators b
            on a.id = b.facility_id;



In [30]:
dfCTGov = pd.read_csv('data/ctgov_investigators.zip', dtype='str')
dfCTGov = dfCTGov.drop(columns=['country'])
dfCTGov.rename(columns={"full_name": "long_name", "postal_code": "postcode"},inplace=True)
dfCTGov.index.name = 'rec_id'
dfCTGov

,long_name,city,state,postcode,nct_id
rec_id,,,,,
0,"André BASCH, MD",Lyon,NaN,NaN,NCT03227419
1,George Philteos,Ajax,Ontario,L1S 2J5,NCT06317285
2,James S. Hu,Los Angeles,California,90020,NCT06422806
3,Bryan A. Faller,Centralia,Illinois,62801,NCT06422806
4,Janet O. Chin,Orland Park,Illinois,60462,NCT06422806
...,...,...,...,...,...
196585,Adam J. Goldrich,Easton,Maryland,21601,NCT05334069
196586,"Aprinda I Queen, PhD",Gainesville,Florida,32610,NCT05998031
196587,"Klaus Arbeiter, MD",Vienna,NaN,1090,NCT01893710


The list of NPI investigators was generated by downloading NPI files from the [Centers fo Medicare & Medicaid Services website](https://download.cms.gov/nppes/NPI_Files.html). The file downloaded was the [Monthly NPPES Downloadable File Version 2 (April 14.2025)](https://download.cms.gov/nppes/NPPES_Data_Dissemination_April_2025_V2.zip). It was a very large file (~10GB), so the first 1000 rows was loaded as a Pandas Dataframe in order to understand it better.

    dfNPI = pd.read_csv("npidata_pfile_20050523-20250413.csv",nrows=1000)
    cols = dfNPI.columns.tolist()

This yielded 330 columns, of which the following were selected for use in further filtering and/or matching:

    cols = ['NPI', 'Entity Type Code', 'Replacement NPI', 'Employer Identification Number (EIN)', 'Provider Organization Name (Legal Business Name)', 'Provider Last Name (Legal Name)', 'Provider First Name', 'Provider Middle Name', 'Provider Name Prefix Text', 'Provider Name Suffix Text', 'Provider Credential Text', 'Provider First Line Business Mailing Address', 'Provider Second Line Business Mailing Address', 'Provider Business Mailing Address City Name', 'Provider Business Mailing Address State Name', 'Provider Business Mailing Address Postal Code', 'Provider Business Mailing Address Country Code (If outside U.S.)', 'Provider Enumeration Date', 'Last Update Date', 'NPI Deactivation Reason Code', 'NPI Deactivation Date', 'NPI Reactivation Date']

Using this list, all data was loaded:

    dfNPI = pd.read_csv("npidata_pfile_20050523-20250413.csv",usecols=cols)

This yielded an 8.8M row dataset. To further reduce the size of the dataset, the following processing steps were applied:

    # removed deactived entries
    dfNPI = dfNPI[dfNPI['NPI Deactivation Date'].isna()]

    # remove orgnization entries by restricting to type code 1
    dfNPI = dfNPI[dfNPI['Entity Type Code'] == 1]

    # process credentials to restrict to only physians (MD or DO)
    dfNPI['creds'] = dfNPI['Provider Credential Text'].str.replace('.','')
    dfNPI = dfNPI[(dfNPI['creds'].str.contains('md', case=False, na=False) | dfNPI['creds'].str.contains('do', case=False, na=False))
                                               & ~dfNPI['creds'].str.contains('pharmd', case=False, na=False)]

This reduced the file down to 1.3M rows. Column names that were to be used for processing were aligned with the column names used through this project and the dataset was trimmed further so that only the columns that matched the columns available from the CTGov sight were extracted and saved into a new CSV file.

    dfNPI = dfNPI.rename(columns={'Provider Last Name (Legal Name)': 'surname', 'Provider First Name':'given_name', 'Provider Business Mailing Address City Name': 'city', 'Provider Business Mailing Address State Name': 'state', 'Provider Business Mailing Address Postal Code': 'postcode', 'NPI': 'npi' })

    dfNPI.to_csv('data/npi_investigators.csv', columns=['given_name', 'surname', 'city', 'state', 'postcode', 'npi'], index=False, header=True)

*NOTE: Code is included for completeness but files were too large to include as part of notebook*



In [31]:
dfNPI = pd.read_csv('data/npi_investigators.zip', dtype='str')
dfNPI['postcode'] = dfNPI['postcode'].str.replace('.0','',regex=False)
dfNPI.index.name = 'rec_id'
dfNPI

,given_name,surname,city,state,postcode,npi
rec_id,,,,,,
0,DAVID,WIEBE,KEARNEY,NE,688482168,1679576722
1,WILLIAM,PILCHER,JACKSONVILLE,FL,322044736,1588667638
2,LAURENT,GRESSOT,HOUSTON,TX,770901243,1215930367
3,RAVI,ADUSUMILLI,TOLEDO,OH,436151753,1932102084
4,ROBERT,BISBEE,LUBBOCK,TX,794073537,1750384806
...,...,...,...,...,...,...
1319114,ARMEND,BALIDEMAJ,BRONX,NY,104611197,1952196685
1319115,ALEXANDER,LE,SUGAR LAND,TX,774786156,1770378408
1319116,JAKE,HUNSAKER,GAINESVILLE,FL,326100254,1689469314


We will use the two 5000 record FEBRL datasets as training datasets and will use the CTGov and NPI datasets as our unknowns and will follow the same steps (Preprocessing, Indexing, Comparison, Classification).

### Preprocessing

Since the ClinicalTrials.gov name field only stores a full name with middle initials and titles at the beginning and end of the name, we needed to use bespoke library called [Nameparser](https://nameparser.readthedocs.io/en/latest/index.html) for name parsing. This was necessary in order to properly split the names into first last and full, which is just first and last.

In [32]:
# force strings to upper case
dfCTGov = dfCTGov.applymap(lambda x: x.upper() if isinstance(x, str) else x)

# create given_name and surname fields from full_name
dfCTGov['long_name'] = dfCTGov['long_name'].fillna('')

constants = Constants()
constants.titles.add("MD-PhD",'pr','msc','otd','msci','b.a.')
constants.suffix_acronyms.add("MD-PhD",'pr','msc','otd','msci','b.a.')

def parse_full_name(long_name):
    name_obj = HumanName(long_name,constants=constants)
    if len(name_obj.first) > 0 and len(name_obj.last) > 0:
        full_name = name_obj.first + ' ' + name_obj.last
    elif len(name_obj.first) == 0 and len(name_obj.last) > 0:
        full_name = name_obj.last
    else:
        full_name = name_obj.first
    return name_obj.first, name_obj.last, full_name

dfCTGov[['given_name','surname', 'full_name']] = dfCTGov['long_name'].apply(lambda x: pd.Series(parse_full_name(x)))

# collapse duplicates
print(f'Size before removing dups: {len(dfCTGov)}')
dfCTGov = dfCTGov.groupby(['given_name','surname','full_name','city','state', 'postcode'], as_index=False).agg({'nct_id': ' '.join})
print(f'Size after removing dups: {len(dfCTGov)}')

# process name fields phonetically
dfCTGov['given_name_ph'] = phonetic(dfCTGov['given_name'],method='metaphone',decode_error='replace')
dfCTGov['surname_ph'] = phonetic(dfCTGov['surname'],method='metaphone',decode_error='replace')

# process city phonetically
dfCTGov[['city', 'state']] = dfCTGov[['city', 'state']].fillna('')
dfCTGov['city_ph'] = phonetic(dfCTGov['city'],method='metaphone',decode_error='replace')

# replace nans in postcode and convert to int
dfCTGov['postcode'] = dfCTGov['postcode'].fillna('0')
dfCTGov['postcode_int'] = pd.to_numeric(dfCTGov['postcode'],errors='coerce')
dfCTGov['postcode_int'] = dfCTGov['postcode_int'].fillna(0)

# build text column
dfCTGov['text'] = dfCTGov['full_name'] + ' ' + dfCTGov['city'] + ' ' + dfCTGov['state']

dfCTGov

Size before removing dups: 196590
Size after removing dups: 77820


,given_name,surname,full_name,city,state,postcode,nct_id,given_name_ph,surname_ph,city_ph,postcode_int,text
0,,,,HOUSTON,TEXAS,77030,NCT05238116,,,HSTN,77030.0,HOUSTON TEXAS
1,,,,NAPLES,FLORIDA,35105,NCT06564311,,,NPLS,35105.0,NAPLES FLORIDA
2,,ABDELNOUR,ABDELNOUR,LOS ANGELES,CALIFORNIA,90095,NCT05268289 NCT05755386,,ABTLNR,LSNJLS,90095.0,ABDELNOUR LOS ANGELES CALIFORNIA
3,,ADAM KRETOWSKI,ADAM KRETOWSKI,BIALYSTOK,PODLASKIE,15-276,NCT04634591,,ATMKRTSK,BLSTK,0.0,ADAM KRETOWSKI BIALYSTOK PODLASKIE
4,,AFONSO NAZÁRIO,AFONSO NAZÁRIO,SÃO PAULO,SP,04004-030,NCT05559528 NCT05398497,,AFNSNSR,SPL,0.0,AFONSO NAZÁRIO SÃO PAULO SP
...,...,...,...,...,...,...,...,...,...,...,...,...
77815,ŠTEFAN,PISTI,ŠTEFAN PISTI,OSTRAVA,MORAVIAN-SILESIAN REGION,728 80,NCT05860387,STFN,PST,OSTRF,0.0,ŠTEFAN PISTI OSTRAVA MORAVIAN-SILESIAN REGION
77816,ŠTEFAN,REGULI,ŠTEFAN REGULI,OSTRAVA,MORAVIAN-SILESIAN REGION,70852,NCT06933199,STFN,RKL,OSTRF,70852.0,ŠTEFAN REGULI OSTRAVA MORAVIAN-SILESIAN REGION
77817,ŠÁRKA,BANÍKOVÁ,ŠÁRKA BANÍKOVÁ,OSTRAVA,MORAVIAN-SILESIAN REGION,70852,NCT05743413,SRK,BNKF,OSTRF,70852.0,ŠÁRKA BANÍKOVÁ OSTRAVA MORAVIAN-SILESIAN REGION
77818,ŠÁRKA,BLAHUTOVÁ,ŠÁRKA BLAHUTOVÁ,OSTRAVA,MORAVIAN-SILESIAN REGION,708 52,NCT06457438,SRK,BLHTF,OSTRF,0.0,ŠÁRKA BLAHUTOVÁ OSTRAVA MORAVIAN-SILESIAN REGION


Processing of the NPI dataset is very similar to the processing of the FEBRL datasets used for training.

In [33]:
# create given_name and surname fields from full_name
dfNPI['full_name'] = dfNPI['given_name'] + ' ' + dfNPI['surname']
dfNPI['full_name'] = dfNPI['full_name'].str.strip()
dfNPI['full_name'] = dfNPI['full_name'].fillna('')

# process name fields phonetically
dfNPI[['given_name', 'surname']] = dfNPI[['given_name', 'surname']].fillna('')
dfNPI['given_name_ph'] = phonetic(dfNPI['given_name'],method='metaphone',decode_error='replace')
dfNPI['surname_ph'] = phonetic(dfNPI['surname'],method='metaphone',decode_error='replace')

# process city phonetically
dfNPI[['city', 'state']] = dfNPI[['city', 'state']].fillna('')
dfNPI['city_ph'] = phonetic(dfNPI['city'],method='metaphone',decode_error='replace')
def convert_abbr_to_name(input):
    state = us.states.lookup(input)
    return state.name.upper() if state else input
dfNPI['state'] = dfNPI['state'].apply(convert_abbr_to_name)

# replace nans in postcode and convert to int
dfNPI['postcode'] = dfNPI['postcode'].fillna('0')
dfNPI['postcode'] = dfNPI['postcode'].str[:5]  # take the first 5 for US matches
dfNPI['postcode_int'] = pd.to_numeric(dfNPI['postcode'],errors='coerce')
dfNPI['postcode_int'] = dfNPI['postcode_int'].fillna(0)

# build text column
dfNPI['text'] = dfNPI['full_name'] + ' ' + dfNPI['city'] + ' ' + dfNPI['state']

dfNPI

,given_name,surname,city,state,postcode,npi,full_name,given_name_ph,surname_ph,city_ph,postcode_int,text
rec_id,,,,,,,,,,,,
0,DAVID,WIEBE,KEARNEY,NEBRASKA,68848,1679576722,DAVID WIEBE,TFT,WB,KRN,68848.0,DAVID WIEBE KEARNEY NEBRASKA
1,WILLIAM,PILCHER,JACKSONVILLE,FLORIDA,32204,1588667638,WILLIAM PILCHER,WLM,PLXR,JKSNFL,32204.0,WILLIAM PILCHER JACKSONVILLE FLORIDA
2,LAURENT,GRESSOT,HOUSTON,TEXAS,77090,1215930367,LAURENT GRESSOT,LRNT,KRST,HSTN,77090.0,LAURENT GRESSOT HOUSTON TEXAS
3,RAVI,ADUSUMILLI,TOLEDO,OHIO,43615,1932102084,RAVI ADUSUMILLI,RF,ATSML,TLT,43615.0,RAVI ADUSUMILLI TOLEDO OHIO
4,ROBERT,BISBEE,LUBBOCK,TEXAS,79407,1750384806,ROBERT BISBEE,RBRT,BSB,LBK,79407.0,ROBERT BISBEE LUBBOCK TEXAS
...,...,...,...,...,...,...,...,...,...,...,...,...
1319114,ARMEND,BALIDEMAJ,BRONX,NEW YORK,10461,1952196685,ARMEND BALIDEMAJ,ARMNT,BLTMJ,BRNKS,10461.0,ARMEND BALIDEMAJ BRONX NEW YORK
1319115,ALEXANDER,LE,SUGAR LAND,TEXAS,77478,1770378408,ALEXANDER LE,ALKSNTR,L,SKRLNT,77478.0,ALEXANDER LE SUGAR LAND TEXAS
1319116,JAKE,HUNSAKER,GAINESVILLE,FLORIDA,32610,1689469314,JAKE HUNSAKER,JK,HNSKR,KNSFL,32610.0,JAKE HUNSAKER GAINESVILLE FLORIDA


In [34]:
# Merge to identify exact matches
merged_df = pd.merge(dfCTGov, dfNPI, on=['given_name', 'surname', 'city', 'state', 'postcode'], how='outer', indicator=True)

# Extract exact matches
dfMatches = merged_df[merged_df['_merge'] == 'both'].drop('_merge', axis=1)
dfMatches = dfMatches[['given_name', 'surname', 'city', 'state', 'postcode', 'npi', 'nct_id']]

# Remove matched records from dfCTGov and dfNPI
dfCTGov = dfCTGov[~dfCTGov.set_index(['given_name', 'surname', 'city', 'state', 'postcode'])
                .index.isin(dfMatches.set_index(['given_name', 'surname', 'city', 'state', 'postcode']).index)].reset_index(drop=True)

dfNPI = dfNPI[~dfNPI.set_index(['given_name', 'surname', 'city', 'state', 'postcode'])
                .index.isin(dfMatches.set_index(['given_name', 'surname', 'city', 'state', 'postcode']).index)].reset_index(drop=True)

# Show exact matches
dfMatches

,given_name,surname,city,state,postcode,npi,nct_id
875,AADEL,CHAUDHURI,ROCHESTER,MINNESOTA,55905,1487097572,NCT06817408
957,AAKASH,BATRA,LOS ANGELES,CALIFORNIA,90027,1578050597,NCT03488693 NCT04852887 NCT06500455 NCT03180268 NCT04804644 NCT05438212 NCT04671667
1027,AAKRITI,SHUKLA,PHILADELPHIA,PENNSYLVANIA,19107,1578976973,NCT04784234
1140,AAMIR,BADRUDDIN,JOLIET,ILLINOIS,60435,1871769331,NCT01748903
1164,AAMIR,JAMAL,SAN DIMAS,CALIFORNIA,91773,1558377523,NCT04557462 NCT06383390
...,...,...,...,...,...,...,...
1388077,ZOE,WEINSTEIN,BOSTON,MASSACHUSETTS,02118,1851682306,NCT06323824
1388293,ZOLTAN,MARI,LAS VEGAS,NEVADA,89106,1366480048,NCT06680830 NCT04477785
1388581,ZUBAID,RAFIQUE,HOUSTON,TEXAS,77030,1730386343,NCT04423198 NCT04423198 NCT05090319
1388635,ZUBAIR,SHAH,KANSAS CITY,KANSAS,66160,1386942126,NCT06526195


In [35]:
dfCTGov

,given_name,surname,full_name,city,state,postcode,nct_id,given_name_ph,surname_ph,city_ph,postcode_int,text
0,,,,HOUSTON,TEXAS,77030,NCT05238116,,,HSTN,77030.0,HOUSTON TEXAS
1,,,,NAPLES,FLORIDA,35105,NCT06564311,,,NPLS,35105.0,NAPLES FLORIDA
2,,ABDELNOUR,ABDELNOUR,LOS ANGELES,CALIFORNIA,90095,NCT05268289 NCT05755386,,ABTLNR,LSNJLS,90095.0,ABDELNOUR LOS ANGELES CALIFORNIA
3,,ADAM KRETOWSKI,ADAM KRETOWSKI,BIALYSTOK,PODLASKIE,15-276,NCT04634591,,ATMKRTSK,BLSTK,0.0,ADAM KRETOWSKI BIALYSTOK PODLASKIE
4,,AFONSO NAZÁRIO,AFONSO NAZÁRIO,SÃO PAULO,SP,04004-030,NCT05559528 NCT05398497,,AFNSNSR,SPL,0.0,AFONSO NAZÁRIO SÃO PAULO SP
...,...,...,...,...,...,...,...,...,...,...,...,...
70035,ŠTEFAN,PISTI,ŠTEFAN PISTI,OSTRAVA,MORAVIAN-SILESIAN REGION,728 80,NCT05860387,STFN,PST,OSTRF,0.0,ŠTEFAN PISTI OSTRAVA MORAVIAN-SILESIAN REGION
70036,ŠTEFAN,REGULI,ŠTEFAN REGULI,OSTRAVA,MORAVIAN-SILESIAN REGION,70852,NCT06933199,STFN,RKL,OSTRF,70852.0,ŠTEFAN REGULI OSTRAVA MORAVIAN-SILESIAN REGION
70037,ŠÁRKA,BANÍKOVÁ,ŠÁRKA BANÍKOVÁ,OSTRAVA,MORAVIAN-SILESIAN REGION,70852,NCT05743413,SRK,BNKF,OSTRF,70852.0,ŠÁRKA BANÍKOVÁ OSTRAVA MORAVIAN-SILESIAN REGION
70038,ŠÁRKA,BLAHUTOVÁ,ŠÁRKA BLAHUTOVÁ,OSTRAVA,MORAVIAN-SILESIAN REGION,708 52,NCT06457438,SRK,BLHTF,OSTRF,0.0,ŠÁRKA BLAHUTOVÁ OSTRAVA MORAVIAN-SILESIAN REGION


In [36]:
dfNPI

,given_name,surname,city,state,postcode,npi,full_name,given_name_ph,surname_ph,city_ph,postcode_int,text
0,DAVID,WIEBE,KEARNEY,NEBRASKA,68848,1679576722,DAVID WIEBE,TFT,WB,KRN,68848.0,DAVID WIEBE KEARNEY NEBRASKA
1,WILLIAM,PILCHER,JACKSONVILLE,FLORIDA,32204,1588667638,WILLIAM PILCHER,WLM,PLXR,JKSNFL,32204.0,WILLIAM PILCHER JACKSONVILLE FLORIDA
2,LAURENT,GRESSOT,HOUSTON,TEXAS,77090,1215930367,LAURENT GRESSOT,LRNT,KRST,HSTN,77090.0,LAURENT GRESSOT HOUSTON TEXAS
3,RAVI,ADUSUMILLI,TOLEDO,OHIO,43615,1932102084,RAVI ADUSUMILLI,RF,ATSML,TLT,43615.0,RAVI ADUSUMILLI TOLEDO OHIO
4,ROBERT,BISBEE,LUBBOCK,TEXAS,79407,1750384806,ROBERT BISBEE,RBRT,BSB,LBK,79407.0,ROBERT BISBEE LUBBOCK TEXAS
...,...,...,...,...,...,...,...,...,...,...,...,...
1311308,ARMEND,BALIDEMAJ,BRONX,NEW YORK,10461,1952196685,ARMEND BALIDEMAJ,ARMNT,BLTMJ,BRNKS,10461.0,ARMEND BALIDEMAJ BRONX NEW YORK
1311309,ALEXANDER,LE,SUGAR LAND,TEXAS,77478,1770378408,ALEXANDER LE,ALKSNTR,L,SKRLNT,77478.0,ALEXANDER LE SUGAR LAND TEXAS
1311310,JAKE,HUNSAKER,GAINESVILLE,FLORIDA,32610,1689469314,JAKE HUNSAKER,JK,HNSKR,KNSFL,32610.0,JAKE HUNSAKER GAINESVILLE FLORIDA
1311311,ZAIN,MAJEED,SCOTTSDALE,ARIZONA,85255,1124813852,ZAIN MAJEED,SN,MJT,SKTSTL,85255.0,ZAIN MAJEED SCOTTSDALE ARIZONA


### Indexing

In this step, we will rely on LSH Blocking of the `full_name` field as it has consistently delivered better performance than exact match

In [37]:
indexer = LSHIndex(column='full_name', threshold=0.6, num_perm=128, tokenizer='bigrams')
pairs_lsh_unknowns = indexer.index(dfCTGov, dfNPI)
pairs_lsh_unknowns
# indexer = recordlinkage.Index()
# indexer.block(left_on='surname_ph', right_on='surname_ph')
# pairs_lsh_unknowns = indexer.index(dfCTGov, dfNPI)
# pairs_lsh_unknowns

Pandas Apply: 100%|██████████| 70040/70040 [01:15<00:00, 928.82it/s]


Seconds to compute minhash dfA: 78.60770729999058


Pandas Apply: 100%|██████████| 1311313/1311313 [24:46<00:00, 882.21it/s]


Seconds to compute minhash dfB: 1489.6861969999736
Seconds to build LSH of dfB: 131.04019839991815
Seconds to query dfB for each row of dfA: 10.861098899971694
Seconds to convert df to MultiIndex: 0.3883300001034513


MultiIndex([(    0,  463491),
            (    0,  725635),
            (    0,  787843),
            (    0, 1218822),
            (    0,  216967),
            (    0, 1247109),
            (    0,  993161),
            (    0, 1176332),
            (    0, 1075469),
            (    0,   70417),
            ...
            (70037, 1018314),
            (70037,  166445),
            (70037,  811757),
            (70037,  233487),
            (70037,   29456),
            (70037,  670515),
            (70037,  428181),
            (70037,  920309),
            (70037,   91262),
            (70037,  531870)],
           length=5120298)

In [38]:
pairs_lsh_unknowns.rename(['dfA_rec_id', 'dfB_rec_id'], inplace=True)
pairs_lsh_unknowns

MultiIndex([(    0,  463491),
            (    0,  725635),
            (    0,  787843),
            (    0, 1218822),
            (    0,  216967),
            (    0, 1247109),
            (    0,  993161),
            (    0, 1176332),
            (    0, 1075469),
            (    0,   70417),
            ...
            (70037, 1018314),
            (70037,  166445),
            (70037,  811757),
            (70037,  233487),
            (70037,   29456),
            (70037,  670515),
            (70037,  428181),
            (70037,  920309),
            (70037,   91262),
            (70037,  531870)],
           names=['dfA_rec_id', 'dfB_rec_id'], length=5120298)

The generation of the above MultiIndex takes about 25m on the reference system described in Appendix 1, so we'll persist it to disk so that we have the option of skipping this step if we choose to on subsequent runs.

In [39]:
df_multiindex = pairs_lsh_unknowns.to_frame()
df_multiindex.to_csv('data/pairs_lsh_unknowns.zip')
# df_multiindex = pd.read_csv('data/pairs_lsh_unknowns.zip')
# pairs_lsh_unknowns = pd.MultiIndex.from_frame(df_multiindex)

### Comparison

Here, we will use the same comparison algorithms but will need to eliminate a few columns that we do not have data for.

In [40]:
comp_ctgov_npi = recordlinkage.Compare()
comp_ctgov_npi.exact(left_on="given_name_ph", right_on="given_name_ph", label="given_name_ph")
comp_ctgov_npi.exact(left_on="surname_ph", right_on="surname_ph", label="surname_ph")
#comp_ctgov_npi.string(left_on="given_name", right_on="given_name", method='jarowinkler', label='given_name')
comp_ctgov_npi.string(left_on="surname", right_on="state", method='jarowinkler', label='surname')
comp_ctgov_npi.exact(left_on="city_ph", right_on="city_ph", label='city_ph')
#comp_ctgov_npi.string(left_on="city", right_on="city", label='city')
comp_ctgov_npi.numeric (left_on="postcode_int", right_on="postcode_int", method='exp', label='postcode')
comp_ctgov_npi.exact(left_on="state", right_on="state", label='state')
f_unknown = comp_ctgov_npi.compute(pairs=pairs_lsh_unknowns, x=dfCTGov, x_link=dfNPI)
f_unknown

given_name_ph  surname_ph   surname  city_ph  postcode  \
dfA_rec_id dfB_rec_id                                                           
0          463491                  1           0  0.000000        0       0.0   
           725635                  1           0  0.000000        0       0.0   
           787843                  0           1  0.000000        0       0.0   
           1218822                 0           1  0.000000        0       0.0   
           216967                  0           1  0.000000        0       0.0   
...                              ...         ...       ...      ...       ...   
70037      670515                  0           0  0.472222        0       0.0   
           428181                  0           0  0.310606        0       0.0   
           920309                  0           0  0.345238        0       0.0   
           91262                   0           0  0.500000        0       0.0   
           531870                  0           0  0.483333        0       0.0   

                       state  
dfA_rec_id dfB_rec_id         
0          463491          0  
           725635          0  
           787843          0  
           1218822         0  
           216967          0  
...                      ...  
70037      670515          0  
           428181          0  
           920309          0  
           91262           0  
           531870          0  

[5120298 rows x 6 columns]

We will be using the FEBRL data generated at the beginning of this workbook as our training data. We will slightly alter `dfA` and `dfB` to align column names to our unknowns data and reprocess comparisons.

In [41]:
dfA_train = dfA.rename(columns={'suburb':'city', 'suburb_ph':'city_ph'})
dfB_train = dfB.rename(columns={'suburb':'city', 'suburb_ph':'city_ph'})
f_train = comp_ctgov_npi.compute(pairs=pairs_lsh, x=dfA_train, x_link=dfB_train)
f_train

given_name_ph  surname_ph   surname  city_ph  \
rec_id_1     rec_id_2                                                       
rec-1070-org rec-2754-dup-0              0           0  0.671429        0   
             rec-2797-dup-0              0           0  0.671429        0   
rec-1016-org rec-1948-dup-0              0           1  0.601190        0   
             rec-865-dup-0               0           0  0.328571        0   
             rec-3267-dup-0              0           1  0.431746        0   
...                                    ...         ...       ...      ...   
rec-1003-org rec-4433-dup-0              0           1  0.416667        0   
rec-4883-org rec-1194-dup-0              0           0  0.566667        0   
             rec-4883-dup-0              1           1  0.566667        0   
             rec-1099-dup-0              0           0  0.377778        0   
rec-66-org   rec-66-dup-0                1           0  0.403704        0   

                                  postcode  state  
rec_id_1     rec_id_2                              
rec-1070-org rec-2754-dup-0   1.274474e-57      0  
             rec-2797-dup-0   0.000000e+00      0  
rec-1016-org rec-1948-dup-0   0.000000e+00      1  
             rec-865-dup-0    6.681912e-52      0  
             rec-3267-dup-0   0.000000e+00      0  
...                                    ...    ...  
rec-1003-org rec-4433-dup-0   0.000000e+00      0  
rec-4883-org rec-1194-dup-0   0.000000e+00      1  
             rec-4883-dup-0   1.000000e+00      1  
             rec-1099-dup-0  5.180654e-318      0  
rec-66-org   rec-66-dup-0     1.000000e+00      1  

[38048 rows x 6 columns]

### Comparison

For our real-world test, we will compare one unsupervised (ECM) and one supeverised (Naive Bayesian Clssifier) learning algorithm with the goal being to match as many of the physicians listed in dfCTGov to physicians in dfNPI.

#### Unsupervised Learning

Since it is not necessary to train ECM, we will simply execute the `fit_predict` method and look at the resulting list of matches.

In [42]:
# ECM using LSH blocking
rw_ecm = recordlinkage.ECMClassifier(binarize=0.97)
rw_result_ecm = rw_ecm.fit_predict(f_unknown)
rw_result_ecm


MultiIndex([(    0,  991469),
            (    2, 1301380),
            (    2,  916911),
            (   22,  495348),
            (   30,  130060),
            (   33,   72524),
            (   65, 1182005),
            (   65, 1165845),
            (   65, 1130272),
            (   65,  968593),
            ...
            (69902,  750146),
            (69902,  103787),
            (69902,  710198),
            (69902,  735543),
            (69920,  743792),
            (69922,  430961),
            (69954,  279034),
            (69960, 1079459),
            (69982,  185368),
            (69983,  185368)],
           names=['dfA_rec_id', 'dfB_rec_id'], length=26063)

In [43]:
dfMatches = rw_result_ecm.to_frame(index=False)
dfCTGov.index.name = 'rec_id'
dfNPI.index.name = 'rec_id'
dfMergeLeft = dfMatches.merge(dfCTGov, left_on='dfA_rec_id', right_on='rec_id', how='inner')
dfMergeRight = dfMatches.merge(dfNPI, left_on='dfB_rec_id', right_on='rec_id', how='inner')
dfFinal = dfMergeLeft.merge(dfMergeRight, on=['dfA_rec_id','dfB_rec_id'], how='inner')
dfFinal.set_index(['dfA_rec_id','dfB_rec_id'], inplace=True)
dfFinal = dfFinal[['npi', 'nct_id', 'given_name_x','surname_x', 'city_x', 'state_x', 'postcode_x', 'given_name_y','surname_y', 'city_y', 'state_y', 'postcode_y']]
dfFinal

npi                   nct_id given_name_x  \
dfA_rec_id dfB_rec_id                                                     
0          991469      1356732119              NCT05238116                
2          1301380     1194527812  NCT05268289 NCT05755386                
           916911      1477817542  NCT05268289 NCT05755386                
22         495348      1437244522              NCT04683653                
30         130060      1730167263              NCT06249555                
...                           ...                      ...          ...   
69922      430961      1437264744              NCT04073563        ZOHER   
69954      279034      1568409662              NCT02012699      ZUBEENA   
69960      1079459     1225559149  NCT05886036 NCT06311227        ZULFA   
69982      185368      1528036779  NCT06331299 NCT04452591          ZVI   
69983      185368      1528036779              NCT06111235          ZVI   

                            surname_x       city_x        state_x postcode_x  \
dfA_rec_id dfB_rec_id                                                          
0          991469                          HOUSTON          TEXAS      77030   
2          1301380          ABDELNOUR  LOS ANGELES     CALIFORNIA      90095   
           916911           ABDELNOUR  LOS ANGELES     CALIFORNIA      90095   
22         495348      ANUJA JHINGRAN      HOUSTON          TEXAS      77030   
30         130060         ATIQUZZAMAN    KISSIMMEE        FLORIDA      34741   
...                               ...          ...            ...        ...   
69922      430961           GHOGAWALA   BURLINGTON  MASSACHUSETTS      01805   
69954      279034              MATEEN      HOLYOKE  MASSACHUSETTS      01040   
69960      1079459               OMER   CINCINNATI           OHIO      45219   
69982      185368           SCHIFFMAN      HOUSTON          TEXAS      77027   
69983      185368           SCHIFFMAN      HOUSTON          TEXAS      77074   

                      given_name_y    surname_y       city_y        state_y  \
dfA_rec_id dfB_rec_id                                                         
0          991469                           LIU      HOUSTON          TEXAS   
2          1301380            MARK    ABDELNOUR  LOS ANGELES     CALIFORNIA   
           916911             LAMA    ABDELNOUR  LOS ANGELES     CALIFORNIA   
22         495348            ANUJA     JHINGRAN      HOUSTON          TEXAS   
30         130060          TAHSINA  ATIQUZZAMAN    KISSIMMEE        FLORIDA   
...                            ...          ...          ...            ...   
69922      430961            ZOHER    GHOGAWALA   BURLINGTON  MASSACHUSETTS   
69954      279034          ZUBEENA       MATEEN      HOLYOKE  MASSACHUSETTS   
69960      1079459           ZULFA         OMER   CINCINNATI           OHIO   
69982      185368              ZVI    SCHIFFMAN      HOUSTON          TEXAS   
69983      185368              ZVI    SCHIFFMAN      HOUSTON          TEXAS   

                      postcode_y  
dfA_rec_id dfB_rec_id             
0          991469          77036  
2          1301380         90036  
           916911          90045  
22         495348          77210  
30         130060          34741  
...                          ...  
69922      430961          18050  
69954      279034          10402  
69960      1079459         45263  
69982      185368          77251  
69983      185368          77251  

[26063 rows x 12 columns]

### Supervised Learning

Here we will train a Naive Bayesian Classifier using all 10k rows provided by the FEBRL dataset. Once the model is trained we will apply to our unknowns and again review results.


In [44]:
#classify using Naive Bayes Classifier
nbc_train = recordlinkage.NaiveBayesClassifier(binarize=0.97)
nbc_train.fit(f_train, miTrueLinks)
nbc_train_result = nbc_train.predict(f_train)
recordlinkage.confusion_matrix(links_true=miTrueLinks, links_pred=nbc_train_result, total=len(dfA.index))

array([[3510, 1490],
       [  34,  -34]])

In [45]:
recordlinkage.fscore(links_true=miTrueLinks, links_pred=nbc_train_result)

0.821629213483146

In [46]:
rw_result_nbc = nbc_train.predict(f_unknown)
dfMatches = rw_result_nbc.to_frame(index=False)
dfMergeLeft = dfMatches.merge(dfCTGov, left_on='dfA_rec_id', right_on='rec_id', how='inner')
dfMergeRight = dfMatches.merge(dfNPI, left_on='dfB_rec_id', right_on='rec_id', how='inner')
dfFinal = dfMergeLeft.merge(dfMergeRight, on=['dfA_rec_id','dfB_rec_id'], how='inner')
dfFinal.set_index(['dfA_rec_id','dfB_rec_id'], inplace=True)
dfFinal = dfFinal[['npi', 'nct_id', 'given_name_x','surname_x', 'city_x', 'state_x', 'postcode_x', 'given_name_y','surname_y', 'city_y', 'state_y', 'postcode_y']]
dfFinal

npi                               nct_id  \
dfA_rec_id dfB_rec_id                                                    
30         130060      1730167263                          NCT06249555   
253        611275      1609910900                          NCT01955148   
307        846282      1073826467                          NCT06065449   
           936100      1154764702                          NCT06065449   
           501153      1700974508                          NCT06065449   
...                           ...                                  ...   
69262      1071612     1326575499                          NCT06921707   
69326      1031092     1730543497                          NCT02418442   
69333      891461      1225300379                          NCT06404086   
69804      885130      1932488368              NCT01629498 NCT05010109   
69849      972154      1619387867  NCT04071457 NCT04759586 NCT04340141   

                      given_name_x       surname_x        city_x  \
dfA_rec_id dfB_rec_id                                              
30         130060                      ATIQUZZAMAN     KISSIMMEE   
253        611275                   MICHAEL S RUMA   ALBUQUERQUE   
307        846282                     QUYNH NGUYEN       HOUSTON   
           936100                     QUYNH NGUYEN       HOUSTON   
           501153                     QUYNH NGUYEN       HOUSTON   
...                            ...             ...           ...   
69262      1071612         ZACHARY          ROWARD       HOUSTON   
69326      1031092           ZANAB            MIAN  LAKE SUCCESS   
69333      891461             ZARA     MARTIROSYAN    WASHINGTON   
69804      885130        ZHONGXING            LIAO       HOUSTON   
69849      972154          ZHUOYAN              LI     BALTIMORE   

                                    state_x postcode_x given_name_y  \
dfA_rec_id dfB_rec_id                                                 
30         130060                   FLORIDA      34741      TAHSINA   
253        611275                NEW MEXICO      87106      MICHAEL   
307        846282                     TEXAS      77030          ANH   
           936100                     TEXAS      77030        QUYEN   
           501153                     TEXAS      77030        PETER   
...                                     ...        ...          ...   
69262      1071612                    TEXAS      78234      ZACHARY   
69326      1031092                 NEW YORK      11042        ZANAB   
69333      891461      DISTRICT OF COLUMBIA      20060         ZARA   
69804      885130                     TEXAS      77030         BING   
69849      972154                  MARYLAND      21204      ZHUOYAN   

                         surname_y               city_y     state_y postcode_y  
dfA_rec_id dfB_rec_id                                                           
30         130060      ATIQUZZAMAN            KISSIMMEE     FLORIDA      34741  
253        611275             RUMA          ALBUQUERQUE  NEW MEXICO      87106  
307        846282           NGUYEN              HOUSTON       TEXAS      77030  
           936100            HUYNH              HOUSTON       TEXAS      77030  
           501153           NGUYEN              HOUSTON       TEXAS      77030  
...                            ...                  ...         ...        ...  
69262      1071612          ROWARD  JBSA FT SAM HOUSTON       TEXAS      78234  
69326      1031092            MIAN        NEW HYDE PARK    NEW YORK      11042  
69333      891461      MARTIROSYAN           WASHINGTON          DC      20060  
69804      885130             LIAO              HOUSTON       TEXAS      77030  
69849      972154               LI               TOWSON    MARYLAND      21204  

[3421 rows x 12 columns]

## Discussion

The results we were able to achieve were promising but still in need of significant additional work. The output of both the ECM and Naive Bayes Classifier (NBC) did yield a significant number of matches, but it was clear that some of the matches identified would be ruled out by a human human reviewer. The most obvious examples of this are the cases where the first name, city and state all matched but the last names did not. Here, both classifiers did not weight the last name score higher than the other values and thus identified a number of matches that clearly are not the same person. As humans, we subconsciously weight last names as a better indicator of matching than other values. Underlying the issue is that both ECM and NBC make the Conditional Independence assumption, which in this case is not accurate.

In examples that were presented with the framework, initial index blocking was done using exact matching of the metaphone of last name. This technique effectively hides/compensates for the weighting of last name at the expense of eliminating possible matches that included a misspelling of the last name that impacted the metaphone. We tried to improve on this by using LSH hashing / Jaccard Similarity of last names to allow for a slightly broader blocking window. In an effort to add weight to last name, we also added a comparison of last name metaphones in addition to string comparison.

More recent work has focused on using LLMs for blocking and neural networks for classification to better compensate for the different weighting of values. Our future work on Record Linkage this will build out a codebase based on LLM and NN.

Finally, it is suspected that better performance from NBS could be achieved if training data was developed from the two real-world datasets. The FEBRL dataset, while convenient, likely does not represent real-world matching problems. A manual matching effort that links a subset of these two datasets would likely have led to better outcomes as well.

## References

Christen, P. (2019). Data linkage: the big picture. Harvard Data Science Review, 1(2). https://doi.org/10.1162/99608f92.84deb5c4

Christen, P (2012). *Data Matching: Concepts and Techniques for Record Linkage, Entity Resolution, and Duplicate Detection*. Springer. DOI 10.1007/978-3-642-31164-2.

Fellegi, I. P., & Sunter, A. B. (1969). A Theory for Record Linkage. Journal of the American Statistical Association, 64(328), 1183. https://doi.org/10.2307/2286061

Winkler, W. (2002). Methods for Record Linkage and Bayesian Networks Methods for Record Linkage and Bayesian Networks. Retrieved May 5, 2025, from https://www.census.gov/content/dam/Census/library/working-papers/2002/adrm/rrs2002-05.pdf

De Bruin, J. [J535D165]. (2023, July 20). Python Record Linkage Toolkit Documentation — Python Record Linkage Toolkit 0.15 documentation. Retrieved April 17, 2025, from https://recordlinkage.readthedocs.io/en/latest/index.html

Dutt, V. (2023, August 1). Understanding Locality-Sensitive hashing for entity matching. Medium. https://medium.com/@mailvdutt/understanding-locality-sensitive-hashing-for-entity-matching-ebed7998c64b

Zhu, E. (Erik) \[ekzhu\]. (2024, June 3). datasketch: Big Data Looks Small — datasketch 1.6.5 documentation. Retrieved May 6, 2025, from https://ekzhu.com/datasketch/index.html

Turk, J. (2023, November 17). Jellyfish. Retrieved May 6, 2025, from https://jamesturk.github.io/jellyfish/

The closer, the better | ElasticSearch: The Definitive Guide [2.x] | Elastic. (n.d.). Elastic. Retrieved May 6, 2025, from https://www.elastic.co/guide/en/elasticsearch/guide/current/decay-functions.html#img-decay-functions

AACT Database | Clinical Trials Transformation Initiative. (2025, April 30). Retrieved May 2, 2025, from https://aact.ctti-clinicaltrials.org/download

NPI files. (2025, April 15). https://download.cms.gov/nppes/NPI_Files.html

Boyanov, M. [mboyanov]. (2020, January 29). Google Colab. Retrieved May 7, 2025, from https://colab.research.google.com/github/mboyanov/minhash-demo/blob/master/Datasketch%20Demo.ipynb#scrollTo=wzZfQeLMeP3N

## Appendix 1: Computing Environment

This is the computing environment used for metrics benchmarks reported in this workbook.

![Computing Environment](images/ReferenceArchitecture.png)

